### Step 1: Data

#### yahoo download

In [2]:
import pandas as pd
import numpy as np
import datetime
import itertools
import yfinance as yf

from yahoodownloader import YahooDownloader
from preprocessors import FeatureEngineer, data_split, INDICATORS
import config_tickers


In [3]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [4]:
aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.449371,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.735306,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.314888,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.970070,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.143906,132079200


In [5]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (20, 8)


In [6]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,73.449402,135480400,aapl,3
1,2020-01-03,74.287498,75.144997,74.125000,72.735306,146322800,aapl,4
2,2020-01-06,73.447502,74.989998,73.187500,73.314880,118387200,aapl,0
3,2020-01-07,74.959999,75.224998,74.370003,72.970085,108872000,aapl,1
4,2020-01-08,74.290001,76.110001,74.290001,74.143898,132079200,aapl,2


In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [9]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.758535,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,43.832630,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.308593,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941105,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,31.579340,7117200,CAT,4


In [12]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


In [16]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [17]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758535,746015200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,2.758535,2.758535,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832630,6547900.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,43.832630,43.832630,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.308593,10955700.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,15.308593,15.308593,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941105,7010200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,33.941105,33.941105,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,31.579340,31.579340,39.189999,0.0


In [18]:
# Split the data for training and trading
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


In [20]:
train.to_csv('train_data.csv', index=False)
trade.to_csv('trade_data.csv', index=False)

### Step 2: Single Model

In [1]:
from zhihu import model, ReplayBuffer, StockEnv

In [8]:
# read the data
train = pd.read_csv('train_data.csv')
trade = pd.read_csv('trade_data.csv')

In [5]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758535,746015200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,2.758535,2.758535,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832630,6547900.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,43.832630,43.832630,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.308593,10955700.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,15.308593,15.308593,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941105,7010200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,33.941105,33.941105,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.0,2.98139,2.652101,100.0,66.666667,100.0,31.579340,31.579340,39.189999,0.0


In [9]:
train['amount'] = train['volume'] * 4
train = train.rename(columns={'day':'adjustflag', 
                      'macd': 'tradestatus',
                      'rsi_30': 'pctChg', 
                      'cci_30': 'peTTM', 
                      'dx_30': 'pbMRQ', 
                      'close_30_sma': 'psTTM'}) 

In [10]:
train

,date,tic,open,high,low,close,volume,adjustflag,tradestatus,boll_ub,boll_lb,pctChg,peTTM,pbMRQ,psTTM,close_60_sma,vix,turbulence,amount
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758535,746015200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,2.758535,2.758535,39.189999,0.000000,2.984061e+09
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832630,6547900.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,43.832630,43.832630,39.189999,0.000000,2.619160e+07
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.308593,10955700.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,15.308593,15.308593,39.189999,0.000000,4.382280e+07
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941105,7010200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,33.941105,33.941105,39.189999,0.000000,2.804080e+07
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,31.579340,31.579340,39.189999,0.000000,2.846880e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83892,2020-06-30,UNH,288.570007,296.450012,287.660004,283.967255,2932900.0,1.0,-0.019218,299.902620,267.660546,52.413022,-25.942826,1.846804,284.207987,277.281662,30.430000,12.918778,1.173160e+07
83893,2020-06-30,V,191.490005,193.750000,190.160004,189.604019,9040100.0,1.0,1.042554,197.569687,183.942009,53.021040,-51.608181,2.013358,190.347365,180.598284,30.430000,12.918778,3.616040e+07
83894,2020-06-30,VZ,54.919998,55.290001,54.360001,47.384804,17414800.0,1.0,-0.411151,50.716149,45.835227,48.097031,-51.247747,8.508886,47.982452,48.408132,30.430000,12.918778,6.965920e+07
83895,2020-06-30,WBA,42.119999,42.580002,41.759998,37.630486,4782100.0,1.0,-0.080962,41.075409,35.173593,48.830188,-14.613148,1.500723,37.726359,37.533500,30.430000,12.918778,1.912840e+07


In [11]:
import numpy as np
import torch
import argparse
import os


# 获得数据
df = train
# df = df.sort_values('date')


# 评估模型
def eval_policy(policy, df, seed, eval_episodes=10):
    # 创建评估环境，并设置随机种子
    eval_env = StockEnv.StockTradingEnv(df)
    eval_env.seed(seed + 100)

    avg_reward = 0.
    for _ in range(eval_episodes):
        # 初始化环境
        state, done = eval_env.reset(), False
        print(state)
        
        # 与环境交互
        while not done:
            action = policy.select_action(state)
            # TODO: step with env
            action[0] *= 3
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward
    
    # 计算平均奖励
    avg_reward /= eval_episodes

    print('-----------------------------------------')
    print(f'Evaluation over {eval_episodes} episodes: {avg_reward:.3f}')
    print('-----------------------------------------')

    return avg_reward


# 默认的超参数
default_seed = 123          # 随机种子
default_batch = 64          # 批量大小
default_gamma = 0.95        # 折扣因子
default_tau = 0.005         # 当前网络参数比例，用于更新目标网络
default_timesteps = 2e5     # 训练步数
default_expl_noise = 0.1    # 高斯噪声
default_eval_freq = 6e3     # 评估模型的频率

if __name__ == '__main__':
    # 路径设置


    # 根据数据集设置环境
    env = StockEnv.StockTradingEnv(df)

    # 设置随机种子
    env.seed(default_seed)

    # T得到环境的参数信息（如：状态和动作的维度）
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[1])
    print(state_dim, action_dim, max_action)

    kwarg = {
        'state_dim': state_dim,
        'action_dim': action_dim,
        "max_action": max_action,
        'gamma': default_gamma,
        'tau': default_tau
    }

    # 设置模型：DDPG算法
    policy = model.DDPGModel(**kwarg)
    
    # 设置缓存容器
    replay_buffer = ReplayBuffer.ReplayBuffer(state_dim, action_dim)

    # 评估初始环境：对照
    evaluations = [eval_policy(policy, df, default_seed)]    

    # 初始化环境
    state, done = env.reset(), False
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0

    # 与环境交互
    for t in range(int(default_timesteps)):

        episode_timesteps += 1

        # 根据状态得到动作
        action = (
            policy.select_action(np.array(state))
            + np.random.normal(0, max_action * default_expl_noise, size=action_dim)
        ).clip(-max_action, max_action)
        action[0] *= 3
        print('action', action)

        # 在环境中执行动作
        next_state, reward, done, _ = env.step(action)
        print('reward', reward)

        # 将交互数据存入容器
        replay_buffer.add(state, action, next_state, reward, done)

        # 状态更新
        state = next_state
        episode_reward += reward

        # 算法训练
        policy.train(replay_buffer, default_batch)

        # 该轮交互结束
        if done:
            # 打印信息，重置状态
            print(f'Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}')
            # Reset environment
            
            state, done = env.reset(), False
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        # 评估算法表现，并存储模型
        if (t + 1) % default_eval_freq == 0:
            evaluations.append(eval_policy(policy, df, default_seed))

        

19 2 1.0
[6.13428593e-04 6.50285816e-04 6.08285809e-04 5.51706982e-04
 7.46015200e-01 9.94686933e+03 4.00000000e-01 0.00000000e+00
 1.00000000e+00 6.66666667e-03 1.00000000e+00 2.75853491e-02
 1.00000000e-01 4.65662078e-02 4.65662078e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]


/Users/jshere/opt/anaconda3/envs/anly591/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


TypeError: tensor(): argument 'dtype' must be torch.dtype, not str